In [ ]:
!pip install geocoder

In [ ]:
!pip install serial

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from twilio.rest import Client
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import pyttsx3
import os
import subprocess
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
from ctypes import cast, POINTER

def send_email(recipient, subject, message):
    sender_email = "patelkelvi23@gmail.com"
    password = "qflw qwgu qtdj bfvn"
    
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, recipient, msg.as_string())
    server.quit()

def send_bulk_email(recipients, subject, message):
    for recipient in recipients:
        send_email(recipient, subject, message)

def send_sms(to, body):
    account_sid = 'AC39e89f6f14011004772d1b45a81ae48b'
    auth_token = 'aa1892eed3181bd2ed1ef8671f71d80d'
    client = Client(account_sid, auth_token)
    
    message = client.messages.create(
        body=body,
        from_='+16189964335',
        to=to
    )
    print(f"Message sent to {to}")

def scrape_google(query):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(f"https://www.google.com/search?q={query}", headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    results = []
    for g in soup.find_all('div', class_='BNeawe vvjwJb AP7Wnd')[:5]:
        results.append(g.get_text())
    
    return results

def get_geo_coordinates():
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode("Your Address")
    return location.latitude, location.longitude, location.address

def text_to_audio(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()


def control_volume(volume_level):
    # Convert volume_level from string to float
    try:
        volume_level = float(volume_level)
    except ValueError:
        print("Please enter a valid number for the volume level.")
        return
    
    # Ensure the volume level is within 0-100
    volume_level = max(0.0, min(100.0, volume_level))
    
    # Convert volume level to a range between 0.0 and 1.0
    volume_level = volume_level / 100.0

    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    # Set the master volume level (0.0 is mute, 1.0 is max)
    volume.SetMasterVolumeLevelScalar(volume_level, None)

    
def main_menu():
    while True:
        print("\n--- Python Automation Tasks Menu ---")
        print("1. Send an Email")
        print("2. Send Bulk Emails")
        print("3. Send an SMS")
        print("4. Scrape Top 5 Google Search Results")
        print("5. Get Current Geo Coordinates and Location")
        print("6. Convert Text to Audio")
        print("7. Control Laptop Volume")
        print("8. Exit")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            recipient = input("Enter recipient email: ")
            subject = input("Enter subject: ")
            message = input("Enter your message: ")
            send_email(recipient, subject, message)
        
        elif choice == '2':
            recipients = input("Enter recipient emails (comma-separated): ").split(',')
            subject = input("Enter subject: ")
            message = input("Enter your message: ")
            send_bulk_email(recipients, subject, message)
        
        elif choice == '3':
            to = input("Enter recipient phone number: ")
            body = input("Enter your message: ")
            send_sms(to, body)
        
        elif choice == '4':
            query = input("Enter search query: ")
            results = scrape_google(query)
            print("Top 5 Google Search Results:")
            for idx, result in enumerate(results):
                print(f"{idx+1}. {result}")
        
        elif choice == '5':
            lat, lon, address = get_geo_coordinates()
            print(f"Latitude: {lat}, Longitude: {lon}")
            print(f"Location: {address}")
        
        elif choice == '6':
            text = input("Enter text to convert to audio: ")
            text_to_audio(text)
        
        elif choice == '7':
            volume_level = input("Enter volume level (0-100): ")
            control_volume(volume_level)
        
        elif choice == '8':
             print("Exiting...")
             break
        
        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main_menu()


In [ ]:
!pip install pycaw

In [ ]:
!pip install geopy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import cv2
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import joblib
import subprocess

def main_menu():
    while True:
        print("\nMain Menu:")
        print("1. Data Processing")
        print("2. Model Integration with Web App")
        print("3. Image Processing Tasks")
        print("4. Exit")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            data_processing()
        elif choice == '2':
            model_integration()
        elif choice == '3':
            image_processing_menu()
        elif choice == '4':
            break
        else:
            print("Invalid choice. Please try again.")

def data_processing():
    file_path = input("Enter the dataset file path (CSV): ")
    df = pd.read_csv(file_path)
    
    # Example data processing: Fill missing values and normalize
    df.fillna(df.mean(), inplace=True)
    normalized_df = (df - df.mean()) / df.std()
    
    print("Data processing completed. Processed data:")
    print(normalized_df.head())

from flask import Flask, request, jsonify
import threading
import requests
import json
import time

# Flask web app
app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the model prediction API!"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json['data']
        data_array = np.array(data).reshape(1, -1)
        prediction = model.predict(data_array)
        return jsonify({'prediction': prediction[0]})
    except Exception as e:
        return jsonify({'error': str(e)})

def run_flask_app():
    app.run(debug=False, use_reloader=False)  # Disable reloader to avoid threading issues on Windows

def model_integration():
    global model  # Declare model as global so it can be accessed by Flask
    file_path = input("Enter the dataset file path (CSV): ")
    df = pd.read_csv(file_path)
    
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Save the trained model
    joblib.dump(model, 'model.pkl')
    print("Model trained and saved successfully.")

    # Start Flask in a separate thread
    threading.Thread(target=run_flask_app).start()
    print("Flask web app started!")

    # Delay to ensure Flask is ready
    time.sleep(2)  # Wait for 2 seconds to ensure the server starts

    # Menu-driven loop for making predictions
    while True:
        print("\n--- Prediction Menu ---")
        print("1. Make a prediction")
        print("2. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            # Making an internal API call to the Flask app
            data_to_predict = input("Enter values for prediction (comma-separated): ")
            data = list(map(float, data_to_predict.split(',')))
            
            try:
                response = requests.post('http://127.0.0.1:5000/predict', json={'data': data})
                print("Response JSON:", response.json())  # Print the entire response to debug
                if response.status_code == 200:
                    print("Prediction:", response.json().get('prediction', "No prediction found"))
                else:
                    print("Error:", response.json())
            except requests.exceptions.ConnectionError:
                print("Error: Could not connect to the Flask API. Please check if the server is running properly.")
        elif choice == '2':
            print("Exiting the program...")
            break
        else:
            print("Invalid choice. Please try again.")



def image_processing_menu():
    while True:
        print("\nImage Processing Menu:")
        print("1. Capture and Crop Face")
        print("2. Apply Filters to Image")
        print("3. Create Custom Image using NumPy")
        print("4. Apply Cool Filters like Sunglasses and Stars")
        print("5. Back to Main Menu")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            capture_and_crop_face()
        elif choice == '2':
            apply_filters_to_image()
        elif choice == '3':
            create_custom_image()
        elif choice == '4':
            apply_cool_filters()
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

def capture_and_crop_face():
    cap = cv2.VideoCapture(0)
    
    print("Press 'c' to capture an image.")
    while True:
        ret, frame = cap.read()
        cv2.imshow('Camera', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Assuming the face is centered, you can use a pre-trained model for better accuracy
    height, width, _ = frame.shape
    cropped_face = frame[height//4:3*height//4, width//4:3*width//4]
    
    cv2.imshow("Cropped Face", cropped_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def apply_filters_to_image():
    cap = cv2.VideoCapture(0)
    
    print("Press 'c' to capture an image.")
    while True:
        ret, frame = cap.read()
        cv2.imshow('Camera', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Apply different filters
    filtered_image = image.filter(ImageFilter.BLUR)
    filtered_image.show()

def create_custom_image():
    # Example: Creating a random image using NumPy
    random_image = np.random.rand(100, 100, 3) * 255
    random_image = random_image.astype('uint8')
    
    plt.imshow(random_image)
    plt.show()

from PIL import Image
import cv2
import numpy as np

def apply_cool_filters():
    # Load the pre-trained face detector from OpenCV (though not necessary for this task)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    cap = cv2.VideoCapture(0)
    
    print("Press 'c' to capture an image.")
    while True:
        ret, frame = cap.read()
        cv2.imshow('Camera', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Load the transparent overlay image
    overlay_image = Image.open("OneDrive/Desktop/sun.png")  # Ensure this image has an alpha channel

    # Convert the captured frame to PIL format for easier manipulation
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Get the dimensions of the image
    img_width, img_height = pil_image.size
    
    # Define the size and position for the overlay image
    overlay_size = (int(img_width * 1), int(img_height * 1))  # Resize overlay image to be 30% of the original image size
    overlay_image = overlay_image.resize(overlay_size, Image.LANCZOS)
    
    # Apply transparency to the overlay image
    overlay_image = overlay_image.convert("RGBA")  # Ensure it's in RGBA mode for alpha channel
    alpha = overlay_image.split()[3]  # Get the alpha channel
    alpha = alpha.point(lambda p: p * 0.2)  # Adjust transparency (0.0 to 1.0)
    overlay_image.putalpha(alpha)  # Put the adjusted alpha channel back

    # Define the position for the overlay image (centered)
    overlay_position = (img_width // 2 - overlay_size[0] // 2, img_height // 2 - overlay_size[1] // 2)
    
    # Paste the overlay image onto the PIL image with transparency support
    pil_image.paste(overlay_image, overlay_position, overlay_image)
    
    # Convert back to OpenCV format
    frame_with_overlay = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    
    # Display the final image with the overlay
    cv2.imshow("Image with Overlay", frame_with_overlay)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



if __name__ == "__main__":
    main_menu()
